# 分类、聚类和异常值改造

使用贝叶斯网络进行分类任务，检测数据集中的异常值，使用层次聚类方法进行聚类分析，使用密度聚类算法发现任意形状的聚类。

## 数据集使用

使用Obesity or CVD risk数据集，该数据集包含了墨西哥、秘鲁和哥伦比亚三个国家中14-61周岁，包括不同的饮食习惯和身体状况的肥胖水平估计。

The data consist of the estimation of obesity levels in people from the countries of Mexico, Peru and Colombia, with ages between 14 and 61 and diverse eating habits and physical condition , data was collected using a web platform with a survey where anonymous users answered each question, then the information was processed obtaining 17 attributes and 2111 records.
The attributes related with eating habits are: Frequent consumption of high caloric food (FAVC), Frequency of consumption of vegetables (FCVC), Number of main meals (NCP), Consumption of food between meals (CAEC), Consumption of water daily (CH20), and Consumption of alcohol (CALC). The attributes related with the physical condition are: Calories consumption monitoring (SCC), Physical activity frequency (FAF), Time using technology devices (TUE), Transportation used (MTRANS)
variables obtained :
Gender, Age, Height and Weight.

NObesity values are:

•Underweight Less than 18.5
•Normal 18.5 to 24.9
•Overweight 25.0 to 29.9
•Obesity I 30.0 to 34.9
•Obesity II 35.0 to 39.9
•Obesity III Higher than 40

The data contains numerical data and continous data, so it can be used for analysis based on algorithms of classification, prediction, segmentation and association. Data is available in CSV format.

## 数据训练过程

**导入软件包**

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.cluster import KMeansClusterer
from sklearn.cluster import DBSCAN
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
import bnlearn as bn
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import HillClimbSearch, BicScore, MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from sklearn.metrics import silhouette_score
from sklearn.cluster import AgglomerativeClustering
from scipy import stats

**导入数据和编码**

In [6]:
# data= pd.read_csv('~/project/Jupyter/data/risk/ObesityDataSet.csv')
data= pd.read_csv('data/risk/ObesityDataSet.csv')

data[data[['family_history_with_overweight','FAVC','SMOKE','SCC']]=='yes']=1
data[data[['family_history_with_overweight','FAVC','SMOKE','SCC']]=='no']=0

data[data[['Gender']]=='Male']=1
data[data[['Gender']]=='Female']=0

data[data[['CAEC','CALC']]=='no']=0
data[data[['CAEC','CALC']]=='Sometimes']=1
data[data[['CAEC','CALC']]=='Frequently']=2
data[data[['CAEC','CALC']]=='Always']=3

data[data[['NObeyesdad']]=='Insufficient_Weight']=0
data[data[['NObeyesdad']]=='Normal_Weight']=1
data[data[['NObeyesdad']]=='Overweight_Level_I']=2
data[data[['NObeyesdad']]=='Overweight_Level_II']=3
data[data[['NObeyesdad']]=='Obesity_Type_I']=4
data[data[['NObeyesdad']]=='Obesity_Type_II']=5
data[data[['NObeyesdad']]=='Obesity_Type_III']=6

data[['family_history_with_overweight','FAVC','SMOKE','SCC','Gender','CAEC','CALC','NObeyesdad']]=data[['family_history_with_overweight','FAVC','SMOKE','SCC','Gender','CAEC','CALC','NObeyesdad']].astype('float64')

MTRANS_encoder=OneHotEncoder(sparse_output=False)
MTRANS_encoder.fit(data[['MTRANS']])
data_MTRANS=MTRANS_encoder.transform(data[['MTRANS']])

MTRANS_df=pd.DataFrame(data_MTRANS,columns=MTRANS_encoder.get_feature_names_out(['MTRANS']))

data.drop(['MTRANS'],axis=1,inplace=True)
data=pd.concat([data,MTRANS_df],axis=1)

y=data['NObeyesdad']

X_train, X_test = train_test_split(data, test_size=0.2, random_state=42)

data.head()
# print(data.info())

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,...,SCC,FAF,TUE,CALC,NObeyesdad,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,0.0,21.0,1.62,64.0,1.0,0.0,2.0,3.0,1.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,0.0,21.0,1.52,56.0,1.0,0.0,3.0,3.0,1.0,1.0,...,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,1.0,23.0,1.80,77.0,1.0,0.0,2.0,3.0,1.0,0.0,...,0.0,2.0,1.0,2.0,1.0,0.0,0.0,0.0,1.0,0.0
3,1.0,27.0,1.80,87.0,0.0,0.0,3.0,3.0,1.0,0.0,...,0.0,2.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0
4,1.0,22.0,1.78,89.8,0.0,0.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,1.0,0.0


**对数据进行贝叶斯网络构建分类**

构建模型和展示

In [7]:
# model=bn.structure_learning.fit(data)
# G=bn.plot(model)
# bn.plot_graphviz(model)
# # bn.structure_scores()

target_column='NObeyesdad'

hc=HillClimbSearch(X_train)
best_model=hc.estimate(scoring_method=BicScore(X_train))

model=BayesianNetwork(best_model.edges())
model.fit(X_train,estimator=MaximumLikelihoodEstimator)

infer=VariableElimination(model)

all_nodes = set(model.nodes())

y_true=X_test[target_column]
y_pred=[]

for index, row in X_test.iterrows():
    # 过滤掉不在贝叶斯网络中的节点
    evidence = {k: v for k, v in row.drop(target_column).to_dict().items() if k in all_nodes}
    prediction = infer.map_query([target_column], evidence=evidence)
    y_pred.append(prediction[target_column])

# 输出分类报告和准确率
print("分类报告：")
print(classification_report(y_true, y_pred))
print("准确率：", accuracy_score(y_true, y_pred))

  0%|          | 0/1000000 [00:00<?, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

分类报告：
              precision    recall  f1-score   support

         0.0       0.50      0.70      0.58        56
         1.0       0.68      0.24      0.36        62
         2.0       0.37      0.18      0.24        56
         3.0       0.47      0.32      0.38        50
         4.0       0.55      0.53      0.54        78
         5.0       0.39      0.74      0.51        58
         6.0       0.81      1.00      0.89        63

    accuracy                           0.54       423
   macro avg       0.54      0.53      0.50       423
weighted avg       0.55      0.54      0.51       423

准确率： 0.5366430260047281


对构建模型进行训练

**对数据进行异常值检测**

In [8]:
# 使用Z分数检测异常值
z_scores = np.abs(stats.zscore(data))
threshold = 3
outliers = np.where(z_scores > threshold)

# 去除异常值
data_cleaned = data[(z_scores < threshold).all(axis=1)]
# 归一化数据
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data_cleaned)
data_scaled=data_cleaned

**对数据应用层次聚类**

In [9]:
# 实施层次聚类
hierarchical_clustering = AgglomerativeClustering(n_clusters=7)
labels_hc = hierarchical_clustering.fit_predict(data_scaled)

# 评估层次聚类
silhouette_hc = silhouette_score(data_scaled, labels_hc)
print(f'Silhouette Score for Hierarchical Clustering: {silhouette_hc}')


Silhouette Score for Hierarchical Clustering: 0.4235212354271746


**对数据应用密度聚类**

In [10]:
# Step 5: 密度聚类
# 实施DBSCAN
dbscan = DBSCAN(eps=4, min_samples=5)
labels_dbscan = dbscan.fit_predict(data_scaled)

# 评估DBSCAN
silhouette_dbscan = silhouette_score(data_scaled, labels_dbscan)
print(f'Silhouette Score for DBSCAN: {silhouette_dbscan}')

# 比较和分析
print(f'Hierarchical Clustering Silhouette Score: {silhouette_hc}')
print(f'DBSCAN Silhouette Score: {silhouette_dbscan}')

Silhouette Score for DBSCAN: 0.03344828942280153
Hierarchical Clustering Silhouette Score: 0.4235212354271746
DBSCAN Silhouette Score: 0.03344828942280153


## 结论

## 使用数据集
[kaggle](https://www.kaggle.com/datasets/aravindpcoder/obesity-or-cvd-risk-classifyregressorcluster)